# PyTorch

- Step 1: Load dataset
- Step 2: Make dataset iterable
- Step 3: Create model class
- Step 4: Instantiate model
- Step 5: Instatiate Loss
- Step 6: Instantiate Optimizer
- Step 7: Train Model

## Imports

In [1]:
import torch
import halper
import collections
import numpy as np
import matplotlib.pyplot as plt

from torch import nn, optim
from torch.autograd import Variable
from torchvision import datasets, transforms

%matplotlib inline
%config InlineBackend.figure_format = 'retina'

## Step 1: Load dataset

In [2]:
# Compose all transformations
transform = transforms.Compose([
    transforms.ToTensor(), 
    transforms.Normalize((0.5,0.5), (0.5,0.5)),
])

# Load training dataset
trainset = datasets.MNIST('~/.pytorch/MNIST_data/',
                          download=True, train=True, transform=transform)

# Load test dataset
testset = datasets.MNIST('~/.pytorch/MNIST_data/',
                          download=True, train=False, transform=transform)

## Step 2: Make dataset iterable

In [3]:
# Make datasets iterable
trainloader = torch.utils.data.DataLoader(trainset, batch_size=100, shuffle=True)
testloader = torch.utils.data.DataLoader(testset, batch_size=100, shuffle=True)

# Test if datasets are iterable
print(isinstance(trainloader, collections.Iterable))
print(isinstance(testloader, collections.Iterable))

True
True


## Step 3: Create model class

In [4]:
class LogisticRegressionModel(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(LogisticRegressionModel, self).__init__()
        self.linear = nn.Linear(input_dim, output_dim)
        
    def forward(self, x):
        out = self.linear(x)
        return out

## Step 4: Instantiate model

In [5]:
dim_input = 28*28
dim_output = 10

model = LogisticRegressionModel(dim_input, dim_output)

## Step 5: Instatiate Loss

In [6]:
criterion = nn.CrossEntropyLoss()

## Step 6: Instantiate Optimizer

In [7]:
optimizer = torch.optim.SGD(model.parameters(), lr=0.001)

## Step 7: Train Model

In [21]:
iterat = 0
epochs = 5

for epoch in range(epochs):
    for images, labels in trainloader:
        
        # Load images and labels
        images = images.view(-1, 28*28)
        
        # Clear the gradient
        optimizer.zero_grad()
        
        # Forward pass
        output = model(images)
        
        # Calculate loss
        loss = criterion(output, labels)
        
        # Update gradients
        loss.backward()
        
        # Update parameters
        optimizer.step()
        
        iterat += 1
        
        if iterat % 500 == 0:
            
            # Calculate accuracy
            correct = 0
            total = 0
            
            with torch.no_grad():
                # Iterate through test dataset
                for images, labels in testloader:

                    # Load images
                    images = images.view(-1, 28*28)

                    # Forward pass
                    outputs = model(images)

                    # Get predictions
                    _, predicted = torch.max(outputs.data, 1)

                    # Total number of labels
                    total += labels.size(0)

                    # Total number of correct predictions
                    correct += (predicted == labels).sum()

                accuracy = 100 * correct / total
            
            # Print loss
            print("Iteration: {}. Loss: {:.5f}. Accuracy: {}.".format(iterat, loss.item(), accuracy))

Iteration: 500. Loss: 0.37916. Accuracy: 90.
Iteration: 1000. Loss: 0.28174. Accuracy: 90.
Iteration: 1500. Loss: 0.38092. Accuracy: 90.
Iteration: 2000. Loss: 0.22705. Accuracy: 91.
Iteration: 2500. Loss: 0.32328. Accuracy: 90.
Iteration: 3000. Loss: 0.38419. Accuracy: 91.


In [22]:
torch.save(model.state_dict(), 'mnist_numbers_model.pkl')